# S-parameter Correction and Manipulation
Examples of how to apply a 12-term and 8-term correction model to s-parameter data. In addition the transformation between S and T matrices. 

In [6]:
from pyMeasure import *
import os
import numpy as np

In [4]:
# import an s-parameter file to play with
os.chdir(TESTS_DIRECTORY)
two_port=S2PV1("20160301_30ft_cable_0.s2p")
two_port.show()

From S to T:
$
T 11   = − det ( S    )    S 21        {\displaystyle T_{11}={\frac {-\det {\begin{pmatrix}S\end{pmatrix}}}{S_{21}}}\,} \\ T_{11}={\frac {-\det {\begin{pmatrix}S\end{pmatrix}}}{S_{21}}}\,\\T 12   = S 11   S 21       \\ {\displaystyle T_{12}={\frac {S_{11}}{S_{21}}}\,}  T_{12}={\frac {S_{11}}{S_{21}}}\,\\T 21   = − S 22    S 21        {\displaystyle T_{21}={\frac {-S_{22}}{S_{21}}}\,} \\ T_{21}={\frac {-S_{22}}{S_{21}}}\,\\T 22   = 1 S 21        {\displaystyle T_{22}={\frac {1}{S_{21}}}\,} \\ T_{22}={\frac {1}{S_{21}}}\,$

In [22]:
# Now create a numpy matrix of sparameters to do T-matrix conversion, should be added to S2PV1 model
matrix_form=[[row[0],np.matrix(row[1:]).reshape((2,2))] for row in two_port.sparameter_complex]

In [53]:
# Convert to T_matrix form, also should be added to S2PV1
t_matrix=[]
for row in matrix_form:
    frequency=row[0]
    m=row[1]
    T11=-np.linalg.det(m)/m[1,0]
    T12=m[0,0]/m[1,0]
    T21=-m[1,1]/m[1,0]
    T22=1/m[1,0]
    t_matrix.append([frequency,np.matrix([[T11,T21],[T21,T22]])])

In [54]:
def S_to_T(S_list):
    """Converts S-parameters into a T Matrix. Input form should be in frequency, np.matrix([[S11,S12],[S21,S22]])
    format. """
    pass


[10.0, matrix([[-0.00165583+0.10131766j, -0.05209200+0.06155311j],
         [-0.05209200+0.06155311j, -0.06457989-9.89974361j]])]

In [50]:
m

matrix([[ 0.00850644+0.00190443j, -0.00109549+0.10089327j],
        [-0.00065892+0.10100842j,  0.00618306+0.00530229j]])

In [52]:
m[1,1]

(0.0061830580648036503+0.0053022888681863401j)